In [1]:
import pysam
bam = "/home/lpryszcz/cluster/rna_mods/N3Pseq/minimap2/N3PS_R9_YeastSequin_TGIRT.bam"
sam = pysam.AlignmentFile(bam)

In [2]:
analyse_first = 250
for a in sam:
    if a.flag&2304: continue
    break

In [4]:
a.is_reverse, a.cigar

(True, [(4, 68), (0, 19), (2, 3), (0, 136), (4, 156)])

In [10]:
idx = -1 if a.is_reverse else 0
clipped = a.cigar[idx][1] if a.cigar[idx][0]==4 else 0; clipped

156

In [12]:
seq = a.get_forward_sequence()[:min(analyse_first, clipped)]; seq

'GTGTACTTCGTTCAGTTACGTACTAAGGTTAACACAAAGACACCGACAACTTTCTTCAGCTTTTCTTCCGTCACTTGCCTGTCAGCTCTATCTTCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT'

In [13]:
a.get_forward_sequence()[clipped-10:clipped+10]

'TTTTTTTTTTTTTTTGTGGG'

### debug sequin

```
20c75d7d-82ac-466b-bb02-db150ac6e954    unknown -10.0   2.5     -1      0       TTCTTTTTTT              78      1.0     39=3S   R1_81_2|6|ext   0       
fab94294-f681-4bc9-bf48-224eaf8a013c    unknown 32.2    2.2     106     116     AGCAGAGGAA      TTCTTTTTTT      78      1.0     39=3S   R1_81_2|6|ext   0       
3ee3820b-8033-4e2c-b745-ab65addadf21    unknown -10.0   1.9     -1      0       ATCAAATTCT              60      0.897   18=1X1=3I16=3S  R1_81_2|6|ext   22      
734ded81-f609-4d63-a478-3f0872bdb5ea    unknown -10.0   2.2     -1      0       CTTTTTTTTT              80      1.0     40=2S   R1_81_2|6|ext   0       
bd7a4e03-68dd-489e-910f-feb38136737d    unknown 33.7    1.9     112     121     GGAGGTTGAA      TTCTTTTTT       76      0.951   36=1D2=1D3=1S   R1_81_2|6|ext   0       
ac01be69-b74c-4cb3-a233-43532356dcf4    unknown -10.0   1.9     -1      0       TTTTTTTTTT              59      0.872   23=2I4=2D5=1I4=3S       R1_81_2|6|ext   0       
4427338f-511d-44c7-ad55-9e417d7210e9    unknown -10.0   2.4     -1      0       CTTTTTTTTT              80      1.0     40=2S   R1_81_2|6|ext   0       
```

In [157]:
import pysam
bam = "/home/lpryszcz/cluster/rna_mods/N3Pseq/minimap2/N3PS_R10_YeastSequin_Induro1.bam"
sam = pysam.AlignmentFile(bam)

In [158]:
readid = "734ded81-f609-4d63-a478-3f0872bdb5ea" # 2
readid = "bba02759-be87-4056-80bb-8a51bb6b427e" # 1
readid = "a2c71395-9648-4a75-8f61-318e8b162d1e"
readid = "1398428a-08f2-4f7b-8461-716b051d8f97"
readid = "806b97b5-c4f2-4982-870e-1ca3ca4203c8"
readid = "7cdd9d6e-4d2b-4403-a9e9-44e9179d1d8e"
for a in sam: 
    if a.flag&2308: continue
    if a.qname==readid: break
print(a)

7cdd9d6e-4d2b-4403-a9e9-44e9179d1d8e	0	#0	43401	60	99S594M1I133M2D32M	*	0	0	TGTTTATGTCACTGTACTCGTATTTGCTAAGGTTAACTTACTACCCAGTGAACCTCCTCGCAGCACCTACTTGCCTGTCGCTCTATCTGCAGAGCAGAGTCATGATTTTTCCAAATAGTTCTTGCAAATGATGGCCATGCAGCTTTGACGGCAAGGACACCCTCTGCGTGGCTGGTATAGAGTTCTTCACCAGTGTTAGGGTCAAGAACAACTGCATCAATACCGAAGAAGGGGAATGAGGCAGAACCCGGTTTCATTGGTGTGACACCACCAGCCAGCGGGGTGACCAGATGCGAACCAGATTCTGTTTGCCAGTAGGTGTCTACAATGGGGATTTCATTTTTACCTATTTTTTCAGAGTACCACTCCCAAACTTCAGCAGCAATTGGTTCACCGACCGAACCCAAGCAACGCAAAGATTTTAAGGAATGATTTTCGATGTAGGAATCACCAGCTCTTTTCAACAAACGCAAAGCAGTTGGGGCAACATAAAATTGGGTGACTTTGTGTTCATCAATAATATCCCAATAACGGGAGTAATTTGGGTACGCAGGAGTCCCTTCAAAGACCAAAGTGGCACAACCATATAGTAAGGGACCATAAACCACATAAGTGTGGCCTGTAATCCAGCCAATGTCTCCAGCTGTGAAGAAAACGTCTTCTTGGTGAGTGTCAAAAGTGTAGCGCATGGTCGAACAAAGCTCCCAGCAAGTAACCTGCGGTAGAATGTTGAACACCCTTGGGGGCACCAGTAGAACCAGACGTATACAACAAGAATAATGGATCCTCAGAATCAACGGGTGTGCATGGATAGTAGGTCTTGTATTTTCTTTTCTGTTGCCCAATCTAAATCTCTGGG	array('B', [6, 7, 12, 10, 6, 5, 5, 7, 11, 9, 8, 7, 8, 8, 9, 10, 

In [159]:
import parasail, re
import numpy as np

primer = "CAGCACCT"+"ACTTGCCTGTCGCTCTATCTGCAGAGCAGAG"+"TTT"
minscore=20
scoring=(2, 3, 3, 2)
match_score, mismatch_penalty, open_penalty, extend_penalty = scoring
matrix = parasail.matrix_create("ACGT", match_score, -mismatch_penalty)
profile = parasail.profile_create_16(primer, matrix)

def correct_alg(cigar, ts, qs, w=6, k=3, pat=re.compile(r'\d+')):
    """Correct alignment skipping initial deletion
    
    Return corrected: 
    - cigar 
    - target start
    - query start
    - number of anchors
    - edit distance (NM)
    - aligned query bases
    """
    idx = n_anchors = nm = qalg = talg = 0
    pe = 0
    diff = w-k
    for i, m in enumerate(pat.finditer(cigar)):
        e = m.end()
        l = int(cigar[pe:e])
        op = cigar[e:e+1]
        pe = e+1
        # skip initial deletion
        if i==0 and op in "ID":
            idx = pe
            if op == "D": ts += l
            elif op == "I": qs += l
        # update number of anchors
        elif op=="=":
            n = (l-diff) // k
            if n>0:
                n_anchors += n
            qalg += l
            talg += l
        # count mismatches
        elif op!="S":
            nm += l
            if op!="D": qalg += l
            if op!="I": talg += l
    return cigar[idx:], ts, qs, n_anchors, nm, qalg, talg

def get_pt_boundaries(seq, pt_start, max_start=10, pat=re.compile(r'((T{1,}\w?)+)', re.IGNORECASE)):
    start = end = 0
    for m in pat.finditer(seq[pt_start:]): # pt_start+after
        s, e = m.span()
        if s>max_start: continue
        if e-s > end - start:
            start, end = s + pt_start, e + pt_start - 1
    return start, end

def alg2pt(a, primer, profile, open_penalty, extend_penalty, extension=5, analyse_first=250, verbose=True):
    """Return sequence flanking pt, pt start, pt end, pt length, pt sequence, alignment score, identity and cigar"""
    # get size of clipped region of the read (from the 5' read end)
    clipped = 0
    if a.cigar:
        idx = -1 if a.is_reverse else 0
        clipped = a.cigar[idx][1] if a.cigar[idx][0]==4 else 0
    if clipped<len(primer):
        return 0, 0, 0, 0, "", "", 0, 0, ""
    # get forward sequence of a read trimming aligned part
    seq = a.get_forward_sequence()[:min(analyse_first, clipped)]
    if verbose: print(seq)
    # find primer in the read sequence
    r = parasail.sw_trace_striped_profile_16(profile, seq, open_penalty, extend_penalty) # 41.6 µs vs 13.4 µs :200
    # filter TODO: scoring
    #if r.score<50: continue
    cigar = r.cigar.decode.decode() # 2.62 µs
    score, ts, te, qs, qe = r.score, r.cigar.beg_ref, r.end_ref, r.cigar.beg_query, r.end_query
    cigar, ts, qs, m, nm, qalg, talg = correct_alg(cigar, ts, qs) # 2.6 µs
    # get polyT length in bases
    end_skip = len(primer)-qe-1
    if qs: cigar = f"{qs}S" + cigar
    if end_skip: cigar += f"{end_skip}S"
    pt_start = te + end_skip - 2
    if verbose: print(te, end_skip, pt_start, seq[pt_start:])
    # get pt boundaries
    s, e = get_pt_boundaries(seq, pt_start-1)
    if verbose: print(s, e, seq[s:e])
    # unload tags
    tags = {k: v for k, v in a.tags}
    identity = round(1-nm/qalg, 3)
    mv = np.array(tags['mv'])
    stride, move = mv[0], mv[1:]
    move_pos = np.argwhere(move==1).flatten()
    steps = move_pos[1:]-move_pos[:-1]
    per_base = steps[e+extension+1:].mean()
    pt_len = round(steps[s-extension:e+extension+1].sum()/per_base - 2*extension, 1)
    #return seq[pt_start-10:pt_start+50], s, e, pt_len, seq[s:e], r.score, identity, cigar
    return pt_len, per_base, s, e, seq[s-10:s], seq[s:e], r.score, identity, cigar

alg2pt(a, primer, profile, open_penalty, extend_penalty)

TGTTTATGTCACTGTACTCGTATTTGCTAAGGTTAACTTACTACCCAGTGAACCTCCTCGCAGCACCTACTTGCCTGTCGCTCTATCTGCAGAGCAGAG
98 3 99 
0 0 


(-10.0, 3.6208920187793425, 0, 0, '', '', 78, 1.0, '39=3S')

In [17]:
# get forward sequence of a read trimming aligned part
seq = a.get_forward_sequence()[:min(analyse_first, clipped)]; seq

'TGTCTACCTCGTTGAGTTACGTATTGCTAAGGTTAAGCATAGTTCTGCATGATGGGTTAGCAGCACCTACTTGCCTGTCGCTCTATCTGCAGAGCAGAGTAAAAAAAAAAAAAAAATTCTTTTTTTTTT'

In [150]:
pat=re.compile(r'((T{2,}\w?)+)', re.IGNORECASE)
seq = "TATTTatg"
pat.findall(seq)
#pat.findall("TATTT")

[('TTTa', 'TTTa')]

In [55]:
pat=re.compile(r'((T{2,}\w\w?|T{1,}\w?)+)', re.IGNORECASE)
pat.findall("TTACTTTTCTATTTTT")

[('TTACTTTTCT', 'TTTTCT'), ('TTTTT', 'TTTTT')]

In [69]:
pat=re.compile(r'((T{2,}\w?)+)', re.IGNORECASE)
pat.findall("TTACTTTTCTATTTTT")

[('TTA', 'TTA'), ('TTTTC', 'TTTTC'), ('TTTTT', 'TTTTT')]

In [52]:
matches = sorted(pat.finditer("TTACTTTTTTTTTT"), key=lambda m: m.end()-m.start())
m = matches[-1]; m.span()

(0, 14)